In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_date, from_unixtime
from pyspark.sql.types import DateType
from pyspark.sql.functions import *
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.ml.feature import *
from operator import add
from pyspark.ml.feature import HashingTF, IDF
from pyspark.ml import Pipeline
from sparknlp.annotator import Tokenizer
from sparknlp.base import DocumentAssembler, Finisher
from pyspark.ml.clustering import KMeans
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings("ignore", category=DeprecationWarning, module="pyspark")
import pandas as pd
from review_preprocessing import *
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

In [2]:
spark

In [2]:
!ln -sf $MKLROOT/lib/intel64/libmkl_rt.so /usr/local/lib/libblas.so.3
!ln -sf $MKLROOT/lib/intel64/libmkl_rt.so /usr/local/lib/liblapack.so.3

In [2]:
import logging
logger = logging.getLogger("org.apache.spark.scheduler.DAGScheduler")
logger.setLevel(logging.ERROR)

In [3]:
path = 'gs://msca-bdp-student-gcs/Group1_Final_Project/amazon_reviews_us_Digital_Music_Purchase_v1_00.tsv'
df = read_trim_tsv(path)
df.show()

Fitting NLP pipeline for column: review_body
Transforming DataFrame for column: review_body
Fitting NLP pipeline for column: review_headline
Transforming DataFrame for column: review_headline
Fitting NLP pipeline for column: product_title
Transforming DataFrame for column: product_title


23/11/25 08:12:51 WARN org.apache.spark.sql.catalyst.util.package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+-----+-----------------+--------------------+--------------------+-----------+---------------+-------------+--------------------+--------------------+-----------------------+---------------------+---------------------+--------------------+----------------+--------------------+--------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes| vine|verified_purchase|     review_headline|         review_body|review_date|headline_length|review_length| review_body_cleaned|   review_body_split|review_headline_cleaned|review_headline_split|product_title_cleaned| product_title_split|review_num_words|       review_3grams|       review_2grams|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+--

In [ ]:
df = vectorize_df(df, features = 40)

23/11/25 08:17:04 WARN com.github.fommil.netlib.LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
23/11/25 08:17:04 WARN com.github.fommil.netlib.LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK


In [ ]:
df.printSchema()
df.show()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: string (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: boolean (nullable = false)
 |-- verified_purchase: boolean (nullable = false)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: date (nullable = true)
 |-- headline_length: integer (nullable = true)
 |-- review_length: integer (nullable = true)
 |-- review_body_cleaned: string (nullable = true)
 |-- review_body_split: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- review_headline_cleaned: string (nullable = true)
 |-- review_headline_split: array (nu

23/11/25 08:37:01 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 3.4 MiB


+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+-----+-----------------+--------------------+--------------------+-----------+---------------+-------------+--------------------+--------------------+-----------------------+---------------------+---------------------+--------------------+----------------+--------------------+--------------------+-------------------------------------+---------------------------------+----------------------------------+------------------------------------------+-----------------------------------------+-------------------------------------+--------------------------------------+----------------------------------------------+---------------------------------------+-----------------------------------+------------------------------------+--------------------------------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_

In [20]:
def feature_assemble(df):
    feature_columns = ['star_rating', 'review_length', 'helpful_votes', 
                       'total_votes', 'verified_purchase',
                       'count_vectorized_review_body_split_reduced', 
                       'count_vectorized_review_headline_split_reduced', 'count_vectorized_product_title_split_reduced',
                       'review_headline_split_tf_idf_features', 'review_headline_split_tf_idf_features',
                       'review_body_split_tf_idf_features']
    # Assemble features into a single vector column
    assembler = VectorAssembler(inputCols=feature_columns, outputCol='features')
    assembled_data = assembler.transform(df)
    return assembled_data;

df.printSchema()
df = feature_assemble(df)
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: string (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: boolean (nullable = false)
 |-- verified_purchase: boolean (nullable = false)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: date (nullable = true)
 |-- headline_length: integer (nullable = true)
 |-- review_length: integer (nullable = true)
 |-- review_body_cleaned: string (nullable = true)
 |-- review_body_split: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- review_headline_cleaned: string (nullable = true)
 |-- review_headline_split: array (nu

In [ ]:
df.select('features').show(5, truncate = False)

23/11/25 08:37:05 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 3.3 MiB


+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
def k_means(df, num_clusters = 5):
    # List of feature columns
    feature_columns = ['star_rating', 'review_length', 'helpful_votes', 
                       'total_votes', 'verified_purchase',
                       'count_vectorized_review_body_split', 
                       'count_vectorized_review_headline_split', 'count_vectorized_product_title_split',
                       'review_headline_split_tf_idf_features', 'review_headline_split_tf_idf_features',
                       'review_body_split_tf_idf_features']
    # Assemble features into a single vector column
    assembler = VectorAssembler(inputCols=feature_columns, outputCol='features')
    assembled_data = assembler.transform(df)
    scaler = StandardScaler(inputCol='features', outputCol='scaled_features')
    scaled_data = scaler.fit(assembled_data).transform(assembled_data)
    kmeans = KMeans(k=num_clusters, featuresCol='scaled_features', predictionCol='cluster')
    model = kmeans.fit(scaled_data)
    result = model.transform(scaled_data)
    return result;

In [ ]:
kmeans_res = k_means(df, num_clusters = 20);
kmeans_res.printSchema();
kmeans_res.head()
df.head()

23/11/21 22:12:34 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 73.6 MiB
23/11/21 22:29:21 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 73.6 MiB
23/11/21 22:29:29 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 79.7 MiB


In [ ]:
kmeans_res.createOrReplaceTempView("reviews")

# SQL query to get 20 clusters and their corresponding reviews
sql_query = """
    SELECT 
        cluster,
        COUNT(cluster)
    FROM reviews
    GROUP BY cluster
"""

# Execute the SQL query
result = spark.sql(sql_query)

# Show the result
result.show(truncate=False)

In [ ]:
# SQL query to get 20 clusters and their corresponding reviews
sql_query = """
    SELECT 
        *
    FROM reviews
    WHERE cluster = 9
"""

# Execute the SQL query
result = spark.sql(sql_query)

# Show the result
result.show(truncate=False)

In [ ]:
path = "gs://msca-bdp-student-gcs/Group1_Final_Project/"
df.write.mode("overwrite").format("parquet").save(path + 'PC.parquet')

23/11/22 04:05:49 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 9.0 MiB


In [11]:
df.printSchema()

root
 |-- id: long (nullable = true)
 |-- features: vector (nullable = true)
 |-- label: double (nullable = true)
 |-- features_vector: vector (nullable = true)



In [3]:
df = spark.read.parquet('gs://msca-bdp-student-gcs/Group1_Final_Project/vectorized_data.parquet')

df.createOrReplaceTempView("df")
query = """
SELECT
    star_rating,
    COUNT(*)
FROM df
GROUP BY star_rating
"""
res = spark.sql(query);
res.show()

+-----------+--------+
|star_rating|count(1)|
+-----------+--------+
|          1| 1668793|
|          3| 1488283|
|          5|10810132|
|          4| 3109991|
|          2|  943670|
+-----------+--------+



In [2]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

def train_logistic_regression(df):
    # Define feature columns
    """feature_columns =  [
        'review_length', 'helpful_votes', 'vine',
        'total_votes', 'verified_purchase',
        'count_vectorized_review_body_split', 
        'count_vectorized_review_headline_split',
        'count_vectorized_product_title_split',
        'review_headline_split_tf_idf_features', 
        'review_body_split_tf_idf_features']

    # Assemble features into a single vector column
    assembler = VectorAssembler(inputCols=feature_columns, outputCol='features')
    assembled_data = assembler.transform(df)"""

    # Select relevant columns
    selected_data = df.select('features', 'star_rating')

    # Split the data into training and testing sets
    train_data, test_data = selected_data.randomSplit([0.8, 0.2], seed=42)
    train_data.printSchema()
    test_data.printSchema()
    # Create a Logistic Regression model
    lr = LogisticRegression(featuresCol='features', labelCol='star_rating')
    print ("LogisticRegression parameters:\n" + lr.explainParams() + "\n")
    # Train the model on the training data
    model = lr.fit(train_data)

    # Make predictions on the test data
    predictions = model.transform(test_data)
    predictions.select('features', 'star_rating', 'prediction').show(100)

    # Evaluate the model
    evaluator = MulticlassClassificationEvaluator(labelCol='star_rating', metricName='accuracy')
    accuracy = evaluator.evaluate(predictions)
    precision = evaluator.evaluate(predictions, {evaluator.metricName: "weightedPrecision"})
    recall = evaluator.evaluate(predictions, {evaluator.metricName: "weightedRecall"})
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")

    return model

# Assuming df is your DataFrame with the given schema
trained_model = train_logistic_regression(df)
print(trained_model)

NameError: name 'df' is not defined

In [4]:
from pyspark.ml.classification import LogisticRegressionModel
trained_model = LogisticRegressionModel.load('gs://msca-bdp-student-gcs/Group1_Final_Project/logistic_regression')

In [27]:
df = trained_model.transform(df)
df.createOrReplaceTempView("df")
query = """
SELECT
    star_rating,
    prediction,
    COUNT(*)
FROM df
GROUP BY star_rating,prediction
"""
res = spark.sql(query);
res.show()

+-----------+----------+--------+
|star_rating|prediction|count(1)|
+-----------+----------+--------+
|          2|       1.0|    6867|
|          4|       3.0|    1330|
|          4|       1.0|       6|
|          2|       4.0|       2|
|          3|       4.0|    1710|
|          5|       4.0|    1314|
|          1|       3.0|      56|
|          3|       1.0|      72|
|          4|       4.0| 3107870|
|          5|       5.0|10808706|
|          5|       1.0|       7|
|          5|       2.0|      19|
|          1|       2.0|    8707|
|          2|       3.0|    1934|
|          3|       2.0|    1867|
|          4|       5.0|     762|
|          3|       3.0| 1484634|
|          2|       2.0|  934867|
|          1|       1.0| 1660030|
|          5|       3.0|      86|
+-----------+----------+--------+
only showing top 20 rows



In [31]:
df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- features: vector (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [5]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator
from pyspark.mllib.evaluation import BinaryClassificationMetrics
# Create an evaluator for binary classification
data = df.select('star_rating', 'features')
predictions = trained_model.transform(data)

# Evaluate the model using MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction", labelCol="star_rating", metricName="f1")

# Calculate the confusion matrix
confusion_matrix = predictions.groupBy("star_rating", "prediction").count()

# Calculate the F1-score
f1_score = evaluator.evaluate(predictions)


Confusion Matrix:


+-----------+----------+--------+
|star_rating|prediction|   count|
+-----------+----------+--------+
|          2|       1.0|    6867|
|          4|       3.0|    1330|
|          2|       4.0|       2|
|          4|       1.0|       6|
|          3|       4.0|    1710|
|          5|       4.0|    1314|
|          1|       3.0|      56|
|          3|       1.0|      72|
|          4|       4.0| 3107870|
|          5|       5.0|10808706|
|          5|       1.0|       7|
|          5|       2.0|      19|
|          1|       2.0|    8707|
|          2|       3.0|    1934|
|          3|       2.0|    1867|
|          4|       5.0|     762|
|          3|       3.0| 1484634|
|          2|       2.0|  934867|
|          1|       1.0| 1660030|
|          5|       3.0|      86|
+-----------+----------+--------+
only showing top 20 rows

F1-Score: 0.9986262097035945


In [6]:
# Show the results
print("Confusion Matrix:")
confusion_matrix.show(100)

print(f"F1-Score: {f1_score}")

Confusion Matrix:


+-----------+----------+--------+
|star_rating|prediction|   count|
+-----------+----------+--------+
|          2|       1.0|    6867|
|          4|       3.0|    1330|
|          4|       1.0|       6|
|          2|       4.0|       2|
|          3|       4.0|    1710|
|          5|       4.0|    1314|
|          1|       3.0|      56|
|          3|       1.0|      72|
|          4|       4.0| 3107870|
|          5|       5.0|10808706|
|          5|       2.0|      19|
|          5|       1.0|       7|
|          1|       2.0|    8707|
|          2|       3.0|    1934|
|          3|       2.0|    1867|
|          4|       5.0|     762|
|          3|       3.0| 1484634|
|          2|       2.0|  934867|
|          1|       1.0| 1660030|
|          5|       3.0|      86|
|          4|       2.0|      23|
+-----------+----------+--------+

F1-Score: 0.9986262097035945


In [11]:
trained_model.save('gs://msca-bdp-student-gcs/Group1_Final_Project/logistic_regression')